# GameBus Health Behavior Mining - Data Extraction

This notebook demonstrates how to extract data from the GameBus platform using the framework.

## Setup

First, let's set up our environment and import the necessary modules.

In [1]:
import sys
import os
import pandas as pd
import json
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Add the project root directory to the Python path
sys.path.append('..')

# Import project modules
from config.credentials import AUTHCODE
from config.paths import USERS_FILE_PATH, RAW_DATA_DIR
from src.extraction.gamebus_client import GameBusClient
from src.extraction.data_collectors import (
    LocationDataCollector, 
    MoodDataCollector,
    ActivityTypeDataCollector,
    HeartRateDataCollector,
    AccelerometerDataCollector,
    NotificationDataCollector
)
from src.utils.logging import setup_logging
from src.utils.file_handlers import load_json, save_json, load_csv, save_csv

## Initialize Logging

Set up logging for the notebook.

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = setup_logging(log_level="INFO")
logger.info("Notebook initialized")

2025-05-30 14:33:51,058 - gamebus_health_mining - INFO - Notebook initialized


INFO:gamebus_health_mining:Notebook initialized


## Load GameBus Users

Load the GameBus users from the CSV file. If the file doesn't exist yet, we'll create it from the example file.

In [3]:
# Load the users
try:
    users_df = pd.read_csv(USERS_FILE_PATH, delimiter=';')
    display(users_df.head())
    print(f"Loaded {len(users_df)} users")
except Exception as e:
    # If users file doesn't exist yet, we need to create it
    print(f"Failed to load users file: {e}")
    print("Loading example users from the examples directory...")
    example_users_path = os.path.join('..', 'examples', 'GB-users.csv')
    users_df = pd.read_csv(example_users_path, delimiter=';')
    # Save to configured path
    os.makedirs(os.path.dirname(USERS_FILE_PATH), exist_ok=True)
    users_df.to_csv(USERS_FILE_PATH, sep=';', index=False)
    display(users_df.head())
    print(f"Loaded {len(users_df)} example users and saved to {USERS_FILE_PATH}")

,Username,Password
0,info+bm@gamebus.nl,password1234
1,m.i.da.graca.jorge.da.silva.ribeiro+test@tue.nl,password1234


Loaded 2 users


## Initialize GameBus Client

Create a GameBus client for API interactions.

In [4]:
# Initialize the GameBus client
client = GameBusClient(AUTHCODE)
print("GameBus client initialized")

GameBus client initialized


## Define Date Range (Optional)

Optionally define a date range for data extraction. If no dates are specified, all available data will be extracted.

In [5]:
# Set to True to enable date filtering
use_date_filter = True

if use_date_filter:
    # Define start date (inclusive)
    start_date = datetime.datetime(2025, 5, 9)  # Format: YYYY, MM, DD
    
    # Define end date (inclusive)
    end_date = datetime.datetime(2025, 6, 9)    # Format: YYYY, MM, DD
    
    print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
else:
    start_date = None
    end_date = None
    print("No date filtering applied - extracting all available data")

Date range: 2025-05-09 to 2025-06-09


## Select User

Select a user from the list to extract data for.

In [6]:
# Select a user
user_index = 0  # Change this to select a different user
selected_user = users_df.iloc[user_index]
username = selected_user['Username']
password = selected_user['Password']

print(f"Selected user: {username}")

Selected user: info+bm@gamebus.nl


## Authenticate User

Authenticate with the GameBus API and get the player token and ID.

In [7]:
# Get player token and ID
token = client.get_player_token(username, password)
player_id = client.get_player_id(token)

print(f"Player token: {token[:10]}..." if token else "Failed to get token")
print(f"Player ID: {player_id}" if player_id else "Failed to get player ID")

DEBUG:src.extraction.gamebus_client:Making POST request to: https://api-new.gamebus.eu/v2/oauth/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api-new.gamebus.eu:443
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "POST /v2/oauth/token HTTP/11" 200 None
INFO:src.extraction.gamebus_client:Token fetched successfully
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/users/current
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/users/current HTTP/11" 200 None
INFO:src.extraction.gamebus_client:Player ID fetched successfully


Player token: eyJhbGciOi...
Player ID: 107631


## Extract GPS Location Data

Extract GPS location data from GameBus. This includes latitude, longitude, altitude, speed, error, timestamp, and arm.

In [8]:
# Create a location data collector
location_collector = LocationDataCollector(client, token, player_id)

# Collect location data
location_data, location_file = location_collector.collect(start_date=start_date, end_date=end_date)
#location_data, location_file = location_collector.collect()
print(f"Collected {len(location_data)} location data points")
print(f"Data saved to {location_file}")

# Display a sample of the data
location_df = pd.DataFrame(location_data)
if len(location_df) > 0:
    display(location_df.head())

INFO:src.extraction.gamebus_client:Filtering data from: 2025-05-09 00:00:00 (timestamp: 1746741600000)
INFO:src.extraction.gamebus_client:Filtering data to: 2025-06-09 00:00:00 (timestamp: 1749420000000)
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=GEOFENCE&from=1746741600000&to=1749420000000&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=GEOFENCE&from=1746741600000&to=1749420000000&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-25 14:03:23
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=GEOFENCE&from=1746741600000&to=1749420000000&page=1


KeyboardInterrupt: 

## Extract Mood Data

Extract mood logging data from GameBus. This includes valence_state_value, arousal_state_value, stress_state_value, and event_timestamp.

In [17]:
# Create a mood data collector
mood_collector = MoodDataCollector(client, token, player_id)

# Collect mood data
#mood_data, mood_file = mood_collector.collect()
mood_data, mood_file = mood_collector.collect(start_date=start_date, end_date=end_date)
print(f"Collected {len(mood_data)} mood data points")
print(f"Data saved to {mood_file}")

# Display a sample of the data
mood_df = pd.DataFrame(mood_data)
if len(mood_df) > 0:
    display(mood_df.head())

INFO:src.extraction.gamebus_client:Filtering data from: 2025-05-09 00:00:00 (timestamp: 1746741600000)
INFO:src.extraction.gamebus_client:Filtering data to: 2025-06-09 00:00:00 (timestamp: 1749420000000)
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=LOG_MOOD&from=1746741600000&to=1749420000000&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=LOG_MOOD&from=1746741600000&to=1749420000000&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-29 10:08:26
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 76 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=LOG_MOOD&from=1746741600000&to=1749420000000&page=1
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v

Collected 76 mood data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_mood.json


,VALENCE_STATE_VALUE,AROUSAL_STATE_VALUE,STRESS_STATE_VALUE,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,6,6,4,1748170800886,3399332,1748506106000,LOG_MOOD
1,3,4,7,1748159734238,3399051,1748472423000,LOG_MOOD
2,3,3,6,1748112467353,3398938,1748455667000,LOG_MOOD
3,7,7,2,1748097923692,3398431,1748374961000,LOG_MOOD
4,6,4,6,1748086645796,3396320,1748343282000,LOG_MOOD


## Extract Activity Type Data

Extract activity type data from GameBus. This includes src, ts (timestamp), type, speed, steps, walks, runs, freq, distance, and cals.

In [18]:
# Create an activity type data collector
activity_collector = ActivityTypeDataCollector(client, token, player_id)

# Collect activity type data
activity_data, activity_file = activity_collector.collect(start_date=start_date, end_date=end_date)
#activity_data, activity_file = activity_collector.collect()

print(f"Collected {len(activity_data)} activity data points")
print(f"Data saved to {activity_file}")

# Display a sample of the data
activity_df = pd.DataFrame(activity_data)
if len(activity_df) > 0:
    display(activity_df.head())

INFO:src.extraction.gamebus_client:Filtering data from: 2025-05-09 00:00:00 (timestamp: 1746741600000)
INFO:src.extraction.gamebus_client:Filtering data to: 2025-06-09 00:00:00 (timestamp: 1749420000000)
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-25 14:02:54
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=1
DEBUG:urllib3.connectionpool:https://ap

Collected 3573 activity data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_activity_type.json


,src,ts,type,speed,steps,walks,runs,freq,distance,cals,activity_id,activity_date
0,p,1748174574156,WALKING,4.1,1707,1693,3,1.6,1254.780029,61.869999,3399729,1748174574000
1,p,1748174574091,UNKNOWN,4.0,1706,1692,3,-1.0,1254.069946,61.840000,3399728,1748174574000
2,p,1748174504011,UNKNOWN,3.9,1602,1586,3,-1.0,1179.630005,57.959999,3399726,1748174504000
3,p,1748174357183,WALKING,4.2,1373,1359,3,1.6,1015.219971,49.709999,3399720,1748174357000
4,p,1748174357109,UNKNOWN,4.1,1373,1359,3,-1.0,1015.219971,49.709999,3399719,1748174357000


## Extract Heart Rate Data

Extract heart rate monitoring data from GameBus. This includes ts (timestamp), hr (heartrate), and pp.

In [19]:
# Create a heart rate data collector
heartrate_collector = HeartRateDataCollector(client, token, player_id)

# Collect heart rate data
heartrate_data, heartrate_file = heartrate_collector.collect(start_date=start_date, end_date=end_date)
#heartrate_data, heartrate_file = heartrate_collector.collect()

print(f"Collected {len(heartrate_data)} heart rate data points")
print(f"Data saved to {heartrate_file}")

# Display a sample of the data
heartrate_df = pd.DataFrame(heartrate_data)
if len(heartrate_df) > 0:
    display(heartrate_df.head())

INFO:src.extraction.gamebus_client:Filtering data from: 2025-05-09 00:00:00 (timestamp: 1746741600000)
INFO:src.extraction.gamebus_client:Filtering data to: 2025-06-09 00:00:00 (timestamp: 1749420000000)
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-25 14:02:54
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=1
DEBUG:urllib3.connectionpool:https://ap

Collected 1433219 heart rate data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_heartrate.json


,ts,hr,pp,activity_id,activity_date
0,1748174417881,90,767,3399723,1748174478000
1,1748174417881,90,767,3399723,1748174478000
2,1748174417888,90,767,3399723,1748174478000
3,1748174418001,90,767,3399723,1748174478000
4,1748174418014,90,767,3399723,1748174478000


## Extract Accelerometer Data

Extract accelerometer data from GameBus. This includes ts (timestamp), x axis, y axis, and z axis.

In [20]:
# Create an accelerometer data collector
accelerometer_collector = AccelerometerDataCollector(client, token, player_id)

# Collect accelerometer data
accelerometer_data, accelerometer_file = accelerometer_collector.collect(start_date=start_date, end_date=end_date)
#accelerometer_data, accelerometer_file = accelerometer_collector.collect()
print(f"Collected {len(accelerometer_data)} accelerometer data points")
print(f"Data saved to {accelerometer_file}")

# Display a sample of the data
accelerometer_df = pd.DataFrame(accelerometer_data)
if len(accelerometer_df) > 0:
    display(accelerometer_df.head())

INFO:src.extraction.gamebus_client:Filtering data from: 2025-05-09 00:00:00 (timestamp: 1746741600000)
INFO:src.extraction.gamebus_client:Filtering data to: 2025-06-09 00:00:00 (timestamp: 1749420000000)
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=0
DEBUG:urllib3.connectionpool:Resetting dropped connection: api-new.gamebus.eu
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from=1746741600000&to=1749420000000&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-25 14:02:54
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&from

Collected 7771993 accelerometer data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_accelerometer.json


,ts,x,y,z,activity_id,activity_date
0,1748174417883,8.463413,-4.654040,0.023928,3399724,1748174478000
1,1748174417883,8.197811,-3.275774,-1.134198,3399724,1748174478000
2,1748174417887,8.331808,-2.703890,-1.172483,3399724,1748174478000
3,1748174417889,8.463413,-2.600998,-0.777667,3399724,1748174478000
4,1748174417895,8.317451,-2.921637,-0.330210,3399724,1748174478000


## Extract Notification Data

Extract notification data from GameBus. This includes action (e.g., received, read) and event_timestamp.

In [21]:
# Create a notification data collector
notification_collector = NotificationDataCollector(client, token, player_id)

# Collect notification data
notification_data, notification_file = notification_collector.collect(start_date=start_date, end_date=end_date)
#notification_data, notification_file = notification_collector.collect()

print(f"Collected {len(notification_data)} notification data points")
print(f"Data saved to {notification_file}")

# Display a sample of the data
notification_df = pd.DataFrame(notification_data)
if len(notification_df) > 0:
    display(notification_df.head())

INFO:src.extraction.gamebus_client:Filtering data from: 2025-05-09 00:00:00 (timestamp: 1746741600000)
INFO:src.extraction.gamebus_client:Filtering data to: 2025-06-09 00:00:00 (timestamp: 1749420000000)
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=NOTIFICATION%28DETAIL%29&from=1746741600000&to=1749420000000&page=0
DEBUG:urllib3.connectionpool:Resetting dropped connection: api-new.gamebus.eu
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=NOTIFICATION%28DETAIL%29&from=1746741600000&to=1749420000000&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-25 12:59:53
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 224 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=NOTIFICA

Collected 158 notification data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_notifications.json


,ACTION,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,READ,1748170793042,3399331,1748170793000,NOTIFICATION(DETAIL)
1,RECEIVED,1748170773754,3399329,1748170774000,NOTIFICATION(DETAIL)
2,RECEIVED,1748170774021,3399330,1748170774000,NOTIFICATION(DETAIL)
3,RECEIVED,1748170773261,3399328,1748170773000,NOTIFICATION(DETAIL)
4,READ,1748159720268,3399049,1748159720000,NOTIFICATION(DETAIL)


## Summary

Let's summarize all the data we've collected.

In [22]:
# Create a summary of collected data
summary = {
    "Location": len(location_data),
    "Mood": len(mood_data),
    "Activity": len(activity_data),
    "Heart Rate": len(heartrate_data),
    "Accelerometer": len(accelerometer_data),
    "Notification": len(notification_data)
}

summary_df = pd.DataFrame({
    "Data Type": list(summary.keys()),
    "Count": list(summary.values())
})

# Display summary
display(summary_df)

print(f"\nTotal data points collected: {sum(summary.values())}")

,Data Type,Count
0,Location,12903
1,Mood,76
2,Activity,3573
3,Heart Rate,1433219
4,Accelerometer,7771993
5,Notification,158



Total data points collected: 9221922
